<a href="https://colab.research.google.com/github/p31x070/GPT/blob/main/PDF_Query_LLM-Adi%207273_MC_DF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://www.youtube.com/watch?v=TLf90ipMzfE

In [ ]:
!pip install langchain
!pip install openai
!pip install PyPDF2
!pip install faiss-cpu
!pip install tiktoken

In [ ]:
from PyPDF2 import PdfReader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import ElasticVectorSearch, Pinecone, Weaviate, FAISS

In [ ]:
# Get your API keys from openai, you will need to create an account. 
# Here is the link to get the keys: https://platform.openai.com/account/billing/overview
import os
os.environ["OPENAI_API_KEY"] = "..."

In [ ]:
# connect your Google Drive
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/My Drive/"

Mounted at /content/gdrive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# location of the pdf file/files. 
reader = PdfReader('/content/drive/MyDrive/Zotero/Informativos STF/MEDIDA CAUTELAR NA AÇÃO DIRETA DE INCONSTITUCIONALIDADE 7.273 DISTRITO FEDERAL/Mendes -  PARTIDO SOCIALISTA BRASILEIRO - PSB NACIONAL  R.pdf')

In [ ]:
reader

In [ ]:
# read data from the file and put them into a variable called raw_text
raw_text = ''
for i, page in enumerate(reader.pages):
    text = page.extract_text()
    if text:
        raw_text += text

# raw_text

In [ ]:
raw_text[:100]

'MEDIDA CAUTELAR NA AÇÃO DIRETA DE INCONSTITUCIONALIDADE  \n7.273 DISTRITO FEDERAL\nRELATOR :MIN. GILMA'

In [ ]:
# We need to split the text that we read into smaller chunks so that during information retreival we don't hit the token size limits. 

text_splitter = CharacterTextSplitter(        
    separator = "\n",
    chunk_size = 1000,
    chunk_overlap  = 200,
    length_function = len,
)
texts = text_splitter.split_text(raw_text)

In [ ]:
len(texts)

71

In [ ]:
texts[0]

'MEDIDA CAUTELAR NA AÇÃO DIRETA DE INCONSTITUCIONALIDADE  \n7.273 DISTRITO FEDERAL\nRELATOR :MIN. GILMAR MENDES\nREQTE.(S):PARTIDO SOCIALISTA BRASILEIRO -  PSB \nNACIONAL \nADV.(A/S):RAFAEL ECHEVERRIA LOPES \nADV.(A/S):RAFAEL DE ALENCAR ARARIPE CARNEIRO \nINTDO.(A/S):CONGRESSO NACIONAL \nPROC.(A/S)(ES):ADVOGADO-GERAL DA UNIÃO \nINTDO.(A/S):PRESIDENTE DA REPÚBLICA \nPROC.(A/S)(ES):ADVOGADO-GERAL DA UNIÃO \nDECISÃO: Trata-se  de  ações  diretas  de  inconstitucionalidade  \npropostas contra o art. 39 da Lei 12.844/2013, em especial seu § 4º, que  \npermite a presunção de legalidade do ouro adquirido e a boa-fé da pessoa  \njurídica adquirente, nos seguintes termos:\n“ Art. 39. A prova da regularidade da primeira aquisição de  \nouro produzido sob qualquer regime de aproveitamento será  \nfeita com base em:\nI  –  nota  fiscal  emitida  por  cooperativa  ou,  no  caso  de  \npessoa física, recibo de venda e declaração de origem do ouro  \nemitido pelo vendedor identificando a área de lavr

In [ ]:
texts[1]

'I  –  nota  fiscal  emitida  por  cooperativa  ou,  no  caso  de  \npessoa física, recibo de venda e declaração de origem do ouro  \nemitido pelo vendedor identificando a área de lavra, o Estado  \nou Distrito Federal e o Município de origem do ouro, o número  \ndo  processo  administrativo  no  órgão  gestor  de  recursos  \nminerais e o número do título autorizativo de extração; e\nII  –  nota  fiscal  de  aquisição  emitida  pela  instituição  \nautorizada pelo Banco Central do Brasil a realizar a compra do  \nouro.\n§ 1º Para os efeitos deste artigo, a instituição legalmente  \nautorizada  a  realizar  a compra  de  ouro  deverá  cadastrar  os  \ndados de identificação do vendedor, tais como nome, número  \nde  inscrição  no  Cadastro  de  Pessoa  Física  do  Ministério  da  \nFazenda – CPF  ou Cadastro Nacional da Pessoa Jurídica do  \nMinistério da Fazenda – CNPJ, e o número de registro no órgão  \nde registro do comércio da sede do vendedor.'

In [ ]:
# Download embeddings from OpenAI
embeddings = OpenAIEmbeddings()

In [ ]:
docsearch = FAISS.from_texts(texts, embeddings)

In [ ]:
docsearch

In [ ]:
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI

In [ ]:
chain = load_qa_chain(OpenAI(), chain_type="stuff")

# Resumo do caso

In [ ]:
query = "Faça um resumo do caso"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

' O Partido Socialista Brasileiro, a Rede Sustentabilidade e o Partido Verde apresentaram uma ação direta de inconstitucionalidade contra o art. 39 da Lei 12.844/2013. O artigo em questão permite a presunção de legalidade do ouro adquirido e a boa-fé da pessoa jurídica adquirente. O juiz decidiu que os requisitos para a concessão da medida cautelar estavam presentes, justificando o deferimento da medida cautelar. Ele também considerou que as alterações promovidas pela Lei de 2013 permitiram a expansão do comércio ilegal, fortalecendo as atividades de garimpo ilegal, desmatamento, contaminação de rios e violência nas regiões de garimpo, atingindo os povos indígenas das áreas afetadas.'

# Norma impugnada

In [ ]:
query = "qual a norma impugnada. transcreva o texto exato do dispositivo"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

' O dispositivo impugnado é o art. 39 da Lei 12.844/2013.'

# Presumem-se a legalidade...

In [ ]:
query = "transcreva o §4 do art. 39 da Lei 12.844/2013."
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

' § 4º Presumem-se a legalidade do ouro adquirido e a boa-fé da pessoa jurídica adquirente quando as informações mencionadas neste artigo, prestadas pelo vendedor, estiverem devidamente arquivadas na instituição legalmente autorizada a realizar a compra de ouro.'

# Sumário

In [ ]:
query = "Escreva uma notícia de Jornal baseada neste voto do Ministro Gilmar Mendes"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

' O Ministro Gilmar Mendes votou em favor da adoção de um marco normativo para a fiscalização do comércio de ouro, visando a verificação da origem legal daquele adquirido por DTVMs. Além disso, o Ministro votou a favor de medidas legislativas, regulatórias e administrativas que inviabilizem a aquisição de ouro extraído de áreas de proteção ambiental e de Terras Indígenas. O Ministro também votou a favor de que a medida cautelar seja julgada no Plenário Virtual, em cumprimento ao disposto no artigo 21, V do RI/STF.'

# O artigo 21, V do RI/STF prevê o referendo

In [ ]:
query = "o que prevê o art 21, V do RI/STF"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

' O artigo 21, V do RI/STF prevê o referendo desta medida cautelar para julgamento no Plenário Virtual.'

In [ ]:
query = "como citar esse esta decisão?"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

' Esta decisão pode ser citada como Adi 7273 MC / DF.'

# palavras chave

In [ ]:
query = "crie palavras chave para o artigo. Apresente o resultado em tópicos"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

' \n- Lei 9.868/1999 \n- Câmara dos Deputados \n- Presidente da República \n- Autoridade Geral da União \n- Subchefia para Assuntos Jurídicos \n- Business Intelligence \n- Sistema Financeiro \n- Ouro Ilegal\n- Tecnologia de Análise de Imagens de Satélite \n- Mandamentos Constitucionais\n- Medida Cautelar'

 
- Lei 9.868/1999 
- Câmara dos Deputados 
- Presidente da República 
- Autoridade Geral da União 
- Subchefia para Assuntos Jurídicos 
- Business Intelligence 
- Sistema Financeiro 
- Ouro Ilegal
- Tecnologia de Análise de Imagens de Satélite 
- Mandamentos Constitucionais
- Medida Cautelar

# Citação - Ouro ilegal

In [ ]:
query = "extraia uma citação do texto que esteja relacionada as palavras chave ouro ilegal. Indique a página"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

' "Sobre a fragilidade provocada na cadeia de controle e rastreabilidade da produção e comércio de ouro oriundo de regiões auríferas gerada com a presunção legal de boa-fé do § 4º do artigo 39 da Lei nº 12.844/2013, o COAF, em discussões da ENCCLA, Estratégia Nacional de Combate à Corrupção e à Lavagem de Dinheiro, a principal rede de articulação institucional brasileira para o arranjo, discussões, formulação e concretização de políticas públicas e soluções de enfrentamento à corrupção e à lavagem de dinheiro, posicionou-se no sentido (página 16).'

# fragilidade do arcabouço normativo

In [ ]:
query = "o que o texto trata sobre: Estratégia Nacional de Combate à Corrupção e à Lavagem de Dinheiro"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

' O texto trata sobre a fragilidade do arcabouço normativo da cadeia produtiva do ouro que estabeleceu uma condição desafiadora para os controles de prevenção à lavagem de dinheiro do Estado. Além disso, menciona que o COAF, a Estratégia Nacional de Combate à Corrupção e à Lavagem de Dinheiro, posicionou-se no sentido de que a possibilidade de controle da origem do metal é muito limitada.'

In [ ]:
query = "relacione os fatos elencados como argumentos para a decisão"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

' A decisão foi baseada na verossimilhança do direito, na relevância da matéria em debate, na discricionariedade legislativa, na inépcia da inicial, na opção legislativa, no espaço de discricionariedade legislativa e nos danos causados pelo garimpo ilegal.'

# Verossimilhança do direito

In [ ]:
query = "o que fundamenta a verossimilhança do direito nesta decisão?"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

' O fundamento da verossimilhança do direito nesta decisão é o princípio da proporcionalidade e a jurisprudência do Supremo Tribunal Federal, que declara a inconstitucionalidade de normas que afastam ou enfraquecem o controle prévio de empreendimentos que impactam o meio ambiente.'

 O fundamento da verossimilhança do direito nesta decisão é o princípio da proporcionalidade e a jurisprudência do Supremo Tribunal Federal, que declara a inconstitucionalidade de normas que afastam ou enfraquecem o controle prévio de empreendimentos que impactam o meio ambiente, de que são exemplo as  Normas que dispensam a exigência de licenciamento para atividades potencialmente poluidoras, como o artigo 10 da Lei 2.713/2013 do Estado do Tocantins.
 

# Princípio da proporcionalidade segundo Canaris.

 O princípio da proporcionalidade pode funcionar como parâmetro de aferição da constitucionalidade das intervenções nos direitos fundamentais como proibições de intervenção. Proibição de excesso (Ubermassverbot) é o princípio de proporcionalidade que proíbe ações que excedam o necessário para se atingir o objetivo desejado. Proibição de proteção deficiente (Untermassverbot), por outro lado, é o princípio de proporcionalidade que veda ações que não protejam o direito fundamental o suficiente ou que não sejam necessárias para a realização desse direito fundamental.

In [ ]:
query = "O Ministro Gilmar Mendes cita Canaris e enuncia o Princípio da Proporcionalidede, segundo diz essse princípio pode ser enunciado como ..."
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

' Proibição de excesso (Ubermassverbot) e proibição de proteção deficiente (Untermassverbot).'

In [ ]:
query = "explique Proibição de excesso (Ubermassverbot) e proibição de proteção deficiente (Untermassverbot) "
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

' O princípio da proporcionalidade pode funcionar como parâmetro de aferição da constitucionalidade das intervenções nos direitos fundamentais como proibições de intervenção. Proibição de excesso (Ubermassverbot) é o princípio de proporcionalidade que proíbe ações que excedam o necessário para se atingir o objetivo desejado. Proibição de proteção deficiente (Untermassverbot), por outro lado, é o princípio de proporcionalidade que veda ações que não protejam o direito fundamental o suficiente ou que não sejam necessárias para a realização desse direito fundamental.'

# Canotilho ensina que o princípio da proibição à proteção insuficiente, em matéria ambiental, retira certa dose de discricionariedade dos poderes públicos, que passam a estar obrigados a agir.

In [ ]:
query = "O princípio da proibição à proteção insuficiente, em matéria ambiental, claramente vincula os Poderes constituídos (Executivo, Legislativo e Judiciário). Qual argumento de de Canotilho é trazido para a discussão do tema?"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

' Canotilho ensina que o princípio da proibição à proteção insuficiente, em matéria ambiental, retira certa dose de discricionariedade dos poderes públicos, que passam a estar obrigados a agir.'

 
Princípio 17 da Declaração de Estocolmo de 1972: "Deve-se confiar às instituições nacionais competentes a tarefa de planejar, administrar ou controlar a utilização dos recursos ambientais dos Estados, com o fim de melhorar a qualidade do meio ambiente." 

Princípio 11 da Declaração do Rio/92: "Os Estados deverão promulgar leis eficazes sobre o meio ambiente."

In [ ]:
query = "O dever de proteção também está previsto na Declaração de Estocolmo, de 1972, e na Declaração da Rio-92. Transcreva cada um deles em tópicos separados."
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

' \nPrincípio 17 da Declaração de Estocolmo de 1972: "Deve-se confiar às instituições nacionais competentes a tarefa de planejar, administrar ou controlar a utilização dos recursos ambientais dos Estados, com o fim de melhorar a qualidade do meio ambiente." \n\nPrincípio 11 da Declaração do Rio/92: "Os Estados deverão promulgar leis eficazes sobre o meio ambiente."'

In [ ]:
query = "O princípio da proibição é citado pelo Ministro Celso de Mello. Transcreva a citação e indique a ADI correspondente."
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

' "respeite o princípio da proporcionalidade, em cuja estrutura normativa compreende-se, além da proibição do excesso, o postulado que veda, em sua outra dimensão, a insuficiência da proteção estatal" (ADI 4901, Tribunal Pleno, Rel. Min. Luiz Fux, voto do Min. Celso de Mello, J. 28.2.2018).'

 "respeite o princípio da proporcionalidade, em cuja estrutura normativa compreende-se, além da proibição do excesso, o postulado que veda, em sua outra dimensão, a insuficiência da proteção estatal" (ADI 4901, Tribunal Pleno, Rel. Min. Luiz Fux, voto do Min. Celso de Mello, J. 28.2.2018).

In [ ]:
query = "Quais órgãos governamentais trouxeram subídios para a causa? faça uma lista em tópicos separados indicando seus posicionamentos."
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

' \n1. Câmara dos Deputados: informou que a matéria encontra-se dentro do espaço de discricionariedade legislativa.\n2. Presidente da República (AGU): alegou a inépcia da inicial e infereu que a lei atacada resultou de uma opção legislativa que desagradou aos requerentes por motivos políticos.\n3. Subchefia para Assuntos Jurídicos: consignou que no âmbito infraconstitucional existe um feixe complexo de atos normativos e portarias ministeriais orientados a regulação, regime de aproveitamento, técnicas e políticas públicas voltadas à preservação ambiental e à vida.\n4. Senado Federal: relata que a Lei 12.844/2013 é resultado da conversão em lei da Medida Provisória 610/2013, que, segundo a exposi'

 
1. Câmara dos Deputados: informou que a matéria encontra-se dentro do espaço de discricionariedade legislativa.
2. Presidente da República (AGU): alegou a inépcia da inicial e infereu que a lei atacada resultou de uma opção legislativa que desagradou aos requerentes por motivos políticos.
3. Subchefia para Assuntos Jurídicos: consignou que no âmbito infraconstitucional existe um feixe complexo de atos normativos e portarias ministeriais orientados a regulação, regime de aproveitamento, técnicas e políticas públicas voltadas à preservação ambiental e à vida.
4. Senado Federal: relata que a Lei 12.844/2013 é resultado da conversão em lei da Medida Provisória 610/2013, que, segundo a exposi

In [ ]:
query = "O que dizem as informações elaboradas pela Secretaria de Acesso à Justiça, do Ministério da Justiça "
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

' As informações elaboradas pela Secretaria de Acesso à Justiça do Ministério da Justiça dizem que os meios eleitos pela norma em questão destoam dos fins que as justificaram, e que é imperioso considerar que a presunção de legalidade está em descompasso com a realidade, pois aponta uma presunção de veracidade justamente em um dos elos mais frágeis da cadeia produtiva do ouro.'

# Argumentos favoráveis à decisão

I. A presunção de legalidade não é compatível com o dever de proteção do meio ambiente, veiculado pelo art. 225 da Constituição Federal (p. 21):

II. A presunção impede a criação de mecanismos de rastreabilidade e responsabilização das DTVMs (p. 21):

III. A Secretaria Nacional de Segurança Pública tem ações de combate ao garimpo ilegal em Terras Indígenas limitadas ao apoio prestado pela Força Nacional de Segurança Pública (p. 22):

IV. O procedimento previsto na legislação impugnada é desprovido de razoabilidade e proporcionalidade, e portanto de lisura constitucional (p. 18).

In [ ]:
query = """ 
Exemplo de argumento favorável a decisâo:

 As informações elaboradas pela Secretaria de Acesso à Justiça do Ministério da Justiça dizem que os meios eleitos pela norma em questão destoam dos fins que as justificaram, e que é imperioso considerar que a presunção de legalidade está em descompasso com a realidade, pois aponta uma presunção de veracidade justamente em um dos elos mais frágeis da cadeia produtiva do ouro.
 
 Pergunta:

 Aponte mais argumentos favoráveis a decisão encontrados no texto indicando a fonte. Divida a resposta em tópicos. Indique a página do documento de onde foram extraídos.

  """
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

'\nI. Argumento de que a presunção de legalidade não é compatível com o dever de proteção do meio ambiente, veiculado pelo art. 225 da Constituição Federal (p. 21):\n\nII. Argumento de que a presunção impede a criação de mecanismos de rastreabilidade e responsabilização das DTVMs (p. 21):\n\nIII. Argumento de que a Secretaria Nacional de Segurança Pública tem ações de combate ao garimpo ilegal em Terras Indígenas limitadas ao apoio prestado pela Força Nacional de Segurança Pública (p. 22):\n\nIV. Argumento de que o procedimento previsto na legislação impugnada é desprovido de razoabilidade e proporcionalidade, e portanto de lisura constitucional (p. 18).'

In [ ]:
query = """ 
Pregunta:

Quais são as consequencias do garimpo para o meio ambiente e povos indigenas apontadas no texto?

Resposta:

Apresente uma lista de conseguencias. 

Formato:

(numero) consequencia.

  """
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

' \n1. Contaminação por mercúrio.\n2. Assoreamento de corpos hídricos.\n3. Supressão de vegetação.\n4. Desmatamento.\n5. Contaminação de rios.\n6. Violência nas regiões de garimpo.\n7. Aumento da violência nos municípios em que o garimpo é ilegal.'

 
1. Contaminação por mercúrio.
2. Assoreamento de corpos hídricos.
3. Supressão de vegetação.
4. Desmatamento.
5. Contaminação de rios.
6. Violência nas regiões de garimpo.
7. Aumento da violência nos municípios em que o garimpo é ilegal.

In [ ]:
query = """ 
Pregunta:

como ocorre o esquentamento do ouro?

Resposta:

Apresente uma lista 

Formato:

(numero) item.

  """
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

'\n\n1. O ouro é adquirido por uma instituição legalmente autorizada a realizar a compra; \n2. É necessário o registro do ouro no Sistema Câmbio do Banco Central; \n3. É necessário que o transporte do ouro seja acompanhado da documentação que indique a suposta origem da extração; \n4. As informações quanto à origem do metal inseridas na documentação da operação são, muitas vezes, falsas;\n5. Não há um rastreamento geoquímico para verificar a origem do ouro.'


1. O ouro é adquirido por uma instituição legalmente autorizada a realizar a compra; 
2. É necessário o registro do ouro no Sistema Câmbio do Banco Central; 
3. É necessário que o transporte do ouro seja acompanhado da documentação que indique a suposta origem da extração; 
4. As informações quanto à origem do metal inseridas na documentação da operação são, muitas vezes, falsas;
5. Não há um rastreamento geoquímico para verificar a origem do ouro.


Resposta útil:

A Estratégia Nacional de Combate à Corrupção e à Lavagem de Dinheiro (ENCCLA) é uma rede de articulação institucional brasileira para o arranjo, discussões, formulação e concretização de políticas públicas e soluções de enfrentamento à corrupção e à lavagem de dinheiro.

In [ ]:
query = """ 
Pregunta:

Defina Estratégia Nacional de Combate à Corrupção e à Lavagem de Dinheiro

Resposta:

definição

Formato:

dissertativo

  """
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

'\n\nA Estratégia Nacional de Combate à Corrupção e à Lavagem de Dinheiro é um projeto de articulação institucional brasileira para o arranjo, discussões, formulação e concretização de políticas públicas e soluções de enfrentamento à corrupção e à lavagem de dinheiro.'

In [ ]:
query = """ 
Pregunta:

apresente uma lista de três conceitos e suas definições extraídos do texto

Resposta:

Conceito definição

Formato:

dissertativo

  """
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

'\nNão se aplica. Este texto é uma decisão judicial. Não há uma lista de três conceitos e suas definições extraídos do texto.'

In [ ]:
query = """ 
Pregunta:

que subsídios a decisão foram trazidos por órgãos governamentais em relação ao ouro?

Resposta:

lista

observações:

incluir consequencias ambientais decorrentes

  """
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

'\nOs órgãos governamentais forneceram informações sobre como o ouro pode ser usado como mercadoria, ativo financeiro e instrumento cambial. O BACEN também esclareceu que é necessário registro no Sistema Câmbio do Banco Central para o ouro ativo financeiro. Além disso, o governo federal também informou que a mineração de ouro tem crescido nos últimos anos em áreas proibidas, como Terras Indígenas ou Unidades de Conservação, o que pode levar a danos ao meio ambiente.'


Os órgãos governamentais forneceram informações sobre como o ouro pode ser usado como mercadoria, ativo financeiro e instrumento cambial. O BACEN também esclareceu que é necessário registro no Sistema Câmbio do Banco Central para o ouro ativo financeiro. Além disso, o governo federal também informou que a mineração de ouro tem crescido nos últimos anos em áreas proibidas, como Terras Indígenas ou Unidades de Conservação, o que pode levar a danos ao meio ambiente.

# É verdadeira a afirmação de que a norma em julgamento favoreceu o crescimento do garimpo e este, por sua vez, contribuiu para o agravamento da questão ambiental?


Sim, é verdadeira a afirmação de que a norma em julgamento favoreceu o crescimento do garimpo ilegal e contribuiu para o agravamento da questão ambiental. Isso é corroborado pelos estudos técnicos, pelas matérias jornalísticas de conhecimento geral e pelas informações do Ministério do Meio Ambiente, da Secretaria de Acesso à Justiça do Ministério da Justiça e pelo estudo dos autores das ações diretas em análise.

In [ ]:
query = """ 
Pregunta:

É verdadeira a afirmação de que a norma em julgamento favoreceu o crescimento do garimpo e este, por sua vez, contribuiu para o agravamento da questão ambiental?

  """
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

'\nSim, é verdadeira a afirmação de que a norma em julgamento favoreceu o crescimento do garimpo ilegal e contribuiu para o agravamento da questão ambiental. Isso é corroborado pelos estudos técnicos, pelas matérias jornalísticas de conhecimento geral e pelas informações do Ministério do Meio Ambiente, da Secretaria de Acesso à Justiça do Ministério da Justiça e pelo estudo dos autores das ações diretas em análise.'

# Verificação de Fatos e Argumentos

In [ ]:
query = """ 
Pregunta:

quais são as alegações trazidas pela Presidência da República em relação ao caso?

  """
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

' A Presidência da República alega a inépcia da inicial e infere que “a lei atacada resultou de uma opção legislativa que desagradou aos requerentes por motivos políticos, fazendo com que a ação fosse ajuizada com a intenção de criticar e desgastar a imagem da atual gestão, sem qualquer efetiva demonstração de inconstitucionalidade”.'

In [ ]:
query = """ 
Pregunta:

as alegações da Presidência da República são suficientes para rebater todos os argumentos favoráveis à decisão proferida? Por que?
  """
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

' Não, as alegações da Presidência da República não são suficientes para rebater todos os argumentos favoráveis à decisão proferida, pois a decisão foi tomada considerando tanto a verossimilhança do direito quanto o perigo na demora. O texto menciona que a norma impugnada foi inserida no processo legislativo e que a matéria encontra-se dentro do espaço de discricionariedade legislativa. Além disso, os requerentes apontam o risco de danos causados pelo garimpo ilegal, o que também foi considerado na decisão.'

In [ ]:
query = """ 
Pregunta:

Quais são as premissas que embasaram a decisão do Ministro para a concessão da liminar?

  """
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

'\n\nAs premissas que embasaram a decisão do Ministro para a concessão da medida liminar foram a plausibilidade jurídica das alegações autorais (fumus boni iuris) e a possibilidade de prejuízo decorrente do retardamento da decisão postulada (periculum in mora).'